In [ ]:
print('Hello World')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime as dt
#os.listdir()


In [ ]:
import requests, zipfile
from io import BytesIO
from io import StringIO

url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/solar/tageswerte_ST_03668_row.zip'

req = requests.get(url)
zip = zipfile.ZipFile(BytesIO(req.content))  #zipfile.ZipFile(BytesIO(req.content))
f = zip.open(zip.namelist()[-1])
content = f.read()
s=str(content,'utf-8')
data = StringIO(s) 

pd.read_csv(data,sep=';',parse_dates=['MESS_DATUM'],usecols=['MESS_DATUM','FD_STRAHL','FG_STRAHL','SD_STRAHL']).head()

In [ ]:
# Stations id's per Bundesland bekommen
url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/solar/ST_Tageswerte_Beschreibung_Stationen.txt'

df_dwd = pd.read_fwf(url,encoding = "ISO-8859-1",colspecs=[(0,5),(6,14),(15,23),(36,38),(43,51),(53,60),(61,84),(142,165)])[1:]
df_dwd.columns = ['Stations_id', 'von_datum', 'bis_datum', 'Stationshoehe', 'geoBreite', 'geoLaenge', 'Stationsname', 'Bundesland']
mask = df_dwd.loc[:,'Bundesland'] == 'Bayern'
list(df_dwd.loc[mask,'Stations_id'])

In [ ]:
# data source: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/solar/
# several stations for BY are missing

df = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})

stationsid = ['03668','05792']
for id in stationsid:
    stationsname = 'Station_'+id+'.txt'
    df_stat = pd.read_csv(stationsname,sep=';',parse_dates=['MESS_DATUM'],usecols=['MESS_DATUM','FD_STRAHL','FG_STRAHL','SD_STRAHL'])
    mask = (df_stat.loc[:,'MESS_DATUM'] >= '2023-01-01') & (df_stat.loc[:,'MESS_DATUM'] <= '2023-12-31')
    df_stat.columns =['date',id+'_FD_STRAHL',id+'_FG_STRAHL',id+'_SD_STRAHL']
    df = df.merge(df_stat.loc[mask,:],on='date')

df.head()

In [ ]:
# data source: https://netztransparenz.tennet.eu/de/strommarkt/transparenz/transparenz-deutschland/netzkennzahlen/tatsaechliche-und-prognostizierte-solarenergieeinspeisung/bayern/
# filtered for 2023-01-01 to 2023-12-31
# input id:"daterange" 

df_en_raw = pd.read_table('solarEnergyFeedIn_BY_2023-01-01_2023-12-31.csv',sep=';',parse_dates=['Datum'],decimal=',')

df_en = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})
df_en.loc[:,'Prog_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Prognostiziert in MW'].sum()).iloc[:,1]
df_en.loc[:,'Act_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Tatsaechlich in MW'].sum()).iloc[:,1]
df = df.merge(df_en,on='date')
df.head()

In [ ]:
#file = open("Einheitensolar/EinheitenSolar_38.xml", "r")
#print(file.read()[:3500])

In [ ]:
# data source: https://download.marktstammdatenregister.de/Gesamtdatenexport_20240306_23.2.zip
# EinheitenSolar_1-39.xml

#import xml.etree.ElementTree as ET
#
#files = list(range(1,40))
#df_mod_raw = pd.DataFrame(columns=['Datum', 'PLZ', 'Bruttoleistung','Nettonennleistung','Inbetriebnahme'])
#counter = 0
#for file in files:
#    source = 'Einheitensolar/EinheitenSolar_'+str(file)+'.xml'
#    tree = ET.parse(source)
#    root = tree.getroot()
#    
#    for einheit in root.findall('EinheitSolar'):
#        try: 
#            land = einheit.find('Bundesland').text
#        except:
#            land = 'Unbekannt'
#        if land == '1403': # Bundesland 1403 = Bayern
#            try: 
#                inbetrieb = einheit.find('Inbetriebnahmedatum').text
#            except:
#                inbetrieb = 'Ausser_Betrieb'
#            
#            if inbetrieb.startswith('2023'):  
#                plz = einheit.find('Postleitzahl').text 
#                brutto = float(einheit.find('Bruttoleistung').text)
#                netto = float(einheit.find('Nettonennleistung').text)
#                datum = einheit.find('DatumLetzteAktualisierung').text
#        
#                datum = datum[:10]
#                df_mod_raw.loc[counter] = [datum,plz,brutto,netto,inbetrieb]
#                counter = counter + 1

In [ ]:
df_mod = pd.DataFrame({'date': pd.date_range(start='1/1/2023', freq='1d', periods=365)})
df_mod.loc[:,'Bruttoleistung'] = pd.DataFrame(df_mod_raw.groupby(['Inbetriebnahme'], as_index=False)['Bruttoleistung'].sum()).iloc[:,1]
df_mod.loc[:,'Nettonennleistung'] = pd.DataFrame(df_mod_raw.groupby(['Inbetriebnahme'], as_index=False)['Nettonennleistung'].sum()).iloc[:,1]
df_mod.loc[:,'Bruttoleistung_kumulativ'] = df_mod.loc[:,'Bruttoleistung'].cumsum()+18643291
df_mod.loc[:,'Nettonennleistung_kumulativ'] = df_mod.loc[:,'Nettonennleistung'].cumsum()+17044680
df = df.merge(df_mod,on='date')
# netto vor 2023: 17.044.680 kW
# brutto vor 2023: 18.643.291 kW

In [ ]:
#df.to_csv('df_BY2023.csv')
df.head()

In [ ]:
df = pd.read_csv('df_final_2015_2023.csv',index_col='Unnamed: 0',parse_dates=['date'])

cols_sd =['03668_SD_STRAHL',
        '05792_SD_STRAHL',
       '05856_SD_STRAHL',
       '05705_SD_STRAHL',
       '02290_SD_STRAHL',
       '05404_SD_STRAHL']
for col in cols_sd:
    df.loc[(df.loc[:,col] < 0),col] = pd.NA

cols_fd_fg =['03668_FD_STRAHL', '03668_FG_STRAHL', 
       '05792_FD_STRAHL', '05792_FG_STRAHL', 
       '05856_FD_STRAHL', '05856_FG_STRAHL', 
       '05705_FD_STRAHL', '05705_FG_STRAHL', 
       '02290_FD_STRAHL', '02290_FG_STRAHL', 
       '05404_FD_STRAHL', '05404_FG_STRAHL']
for col in cols_fd_fg:
    df.loc[(df.loc[:,col] < 1),col] = pd.NA

df.plot(x='date',y='05792_FD_STRAHL',figsize=(10,6)) #

# netto vor 2015: 10.135.888 kW 
# brutto vor 2015: 10.941.816 kW
df.loc[:,'Bruttoleistung_kumulativ'] = df.loc[:,'Bruttoleistung'].cumsum()+10941816
df.loc[:,'Nettoleistung_kumulativ'] = df.loc[:,'Nettoleistung'].cumsum()+10135888

In [ ]:
#display(df.loc[(df['date'].dt.day==1) & (df['date'].dt.month==1),['05792_FD_STRAHL']])#.median() 
#display(df.loc[(df.index==365) & (df['date'].dt.day==1) & (df['date'].dt.month==1),['03668_FD_STRAHL','05792_FD_STRAHL','05856_FD_STRAHL','05705_FD_STRAHL','02290_FD_STRAHL']])#.median() #

In [ ]:
import numpy as np
r, c = np.where(df.iloc[:,:-4].isna())
print(r[0],c[0])
#print(len(r),len(c))
print(df.iloc[r[0],c[0]])
cols_na = [col for col in df.columns if col.endswith(str(df.columns[c[0]])[5:])]
cols_na

In [ ]:
#display(df.loc[(df['date'].dt.day==(df.iloc[31,0].day)) & (df['date'].dt.month==(df.iloc[31,0].month)),df.columns[c[0]]])
#display(df.loc[ (df['date'].dt.day==(df.iloc[31,0].day)) & (df['date'].dt.month==(df.iloc[31,0].month)),cols_na]) #(df.index==31) &

In [ ]:
for i in range(len(r)):
    cols_na = [col for col in df.columns if col.endswith(str(df.columns[c[i]])[5:])]
    med_col = df.loc[(df['date'].dt.day==(df.iloc[r[i],0].day)) & (df['date'].dt.month==(df.iloc[r[i],0].month)),df.columns[c[i]]].median()
    
    med_row = df.loc[(df.index==r[i]) & (df['date'].dt.day==(df.iloc[r[i],0].day)) & (df['date'].dt.month==(df.iloc[r[i],0].month)),cols_na].sum().median()
    mean_v = (med_col+med_row)/2
    #print(mean_v)
    df.iloc[r[i],c[i]] =mean_v

df.plot(x='date',y='05792_FD_STRAHL',figsize=(10,6))

In [ ]:
df.describe()
df.to_csv('us_df_final_2015_2023.csv')

In [ ]:
df.plot(x='date',y='Act_in_MW',figsize=(10,6))

In [ ]:
df.plot(x='date',y='Bruttoleistung_kumulativ',figsize=(10,6)) 